# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate work to the Data Analyst to continuously monitor and analyze market data for AAPL, as well as ask the Trading Strategy Developer to use statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The Data Analyst needs to collect, analyze, and interpret market data specifically for the stock AAPL. This includes looking at historical data, current trends, and any relevant news or events that may impact the stock price."
}


> Entering new CrewAgentExecutor chain...
I need to gather as much relevant i

Final Answer: Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products including iPhone, Mac, iPad, and various wearables and accessories. Additionally, Apple provides AppleCare support, cloud services, and operates platforms like the App Store. The company also offers subscription-based services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Pay. Apple serves consumers, small and mid-sized businesses, education, enterprise, and government markets. The company was founded in 1976 and is headquartered in Cupertino, California.

> Finished chain.
 

Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products including iPhone, Mac, iPad, and various wearables and accessories. Additionally, Apple provides AppleCare support, cloud

I need to delegate tasks to the Data Analyst and Trading Strategy Developer to develop and refine trading strategies for AAPL based on insights and user-defined risk tolerance.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Provide insights on AAPL market data and performance", "context": "Apple Inc. (AAPL) is a leading technology company that designs, manufactures, and markets a range of products worldwide. The Data Analyst will need to continuously monitor and analyze market data for AAPL to provide valuable insights."}

> Entering new CrewAgentExecutor chain...
I need to gather the most up-to-date information on AAPL market data and performance to provide valuable insights.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Final Answer:
I'm sorry, but I was unable to retrieve the necessary information from the provided w

Thought: I need to delegate specific tasks to the Data Analyst and Trading Strategy Developer to gather insights and refine trading strategies for AAPL based on the provided context.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Provide insights on AAPL market data and performance", "context": "Apple Inc. (AAPL) is a leading technology company that designs, manufactures, and markets a range of products worldwide. The Data Analyst will need to continuously monitor and analyze market data for AAPL to provide valuable insights."} 

I'm sorry, but I was unable to retrieve the necessary information from the provided website URL. Would you like me to try a different source or approach to gather the AAPL market data and performance insights?

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "What specific market data points are you currently monitoring for AAPL?", "context": "The Data Analyst will need to contin

Final Answer: 
Based on my search, I found various resources on machine learning models for stock price prediction for AAPL. Some of the key findings include:
- GitHub repository containing Python code for stock price prediction using various Machine Learning models.
- Article on predicting Apple stock prices using an LSTM model with TensorFlow.
- Study focusing on creating a dataset for AAPL stock prices and forecasting closing prices with a Random Forest Regressor.
- Comparative analysis of machine learning models like Random Forest, Logistic Regression, Support Vector Machine, and Gradient Descent for stock price prediction.
- Article on using traditional quantitative finance methodology and machine learning algorithms to predict stock movements.

It seems that utilizing machine learning models, particularly LSTM and Random Forest Regressor, could be effective for developing trading strategies for AAPL stock price predictions. Further analysis and testing of these models could help 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

As the Crew Manager, I have delegated the task of evaluating the risks associated with the proposed trading strategies and execution plans for AAPL to the Risk Advisor. The Risk Advisor will provide a comprehensive risk analysis report detailing potential risks and mitigation recommendations based on the strategies outlined.